### Importing necessary libraries:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Problem Statement:

Have you ever wondered how lenders use various factors such as credit score, annual income, the loan amount approved, tenure, debt-to-income ratio etc. and select your interest rates? 

The process, defined as ‘risk-based pricing’, uses a sophisticated algorithm that leverages different determining factors of a loan applicant. Selection of significant factors will help develop a prediction algorithm which can estimate loan interest rates based on clients’ information. On one hand, knowing the factors will help consumers and borrowers to increase their credit worthiness and place themselves in a better position to negotiate for getting a lower interest rate. On the other hand, this will help lending companies to get an immediate fixed interest rate estimation based on clients information. Here, your goal is to use a training dataset to predict the loan rate category (1 / 2 / 3) that will be assigned to each loan in our test set.

### Reading train and test dataset:

In [ ]:
train_set=pd.read_csv('../input/banking-modelclassification/train_loan.csv')
train_set

In [ ]:
test_set=pd.read_csv('../input/banking-modelclassification/test_loan.csv')

In [ ]:
test_set

In [ ]:
train_set.shape,test_set.shape

In [ ]:
train_set['Data']='train'
test_set['Data']='test'
test_set['Interest_Rate']=np.nan

In [ ]:
combined=pd.concat([train_set,test_set],ignore_index=True,sort=False)

In [ ]:
combined

In [ ]:
combined.shape

In [ ]:
combined.info()

### Checking null values:

In [ ]:
combined.isnull().sum()[combined.isnull().sum()!=0]

### Data Cleaning & Imputing null values:

In [ ]:
combined.head()

In [ ]:
combined['Loan_Amount_Requested'].value_counts()

In [ ]:
combined['Loan_Amount_Requested']=combined['Loan_Amount_Requested'].str.replace(',','')

In [ ]:
combined.info()

In [ ]:
combined['Loan_Amount_Requested'].unique()

In [ ]:
combined['Loan_Amount_Requested']=pd.to_numeric(combined['Loan_Amount_Requested'],errors='coerce')

In [ ]:
combined.isnull().sum()[combined.isnull().sum()!=0]

In [ ]:
combined.info()

In [ ]:
combined.head()

In [ ]:
combined['Length_Employed']=combined['Length_Employed'].str.replace('<','')

In [ ]:
combined['Length_Employed'].value_counts()

In [ ]:
combined['Length_Employed']=combined['Length_Employed'].str.replace('+','')

In [ ]:
combined['Length_Employed']=combined['Length_Employed'].str.replace('years','')

In [ ]:
combined['Length_Employed'].value_counts()

In [ ]:
combined['Length_Employed']=combined['Length_Employed'].str.replace('year','')

In [ ]:
combined['Length_Employed'].value_counts()

In [ ]:
combined['Length_Employed']=pd.to_numeric(combined['Length_Employed'],errors='coerce')

In [ ]:
combined.isnull().sum()[combined.isnull().sum()!=0]

In [ ]:
combined.info()

In [ ]:
combined['Home_Owner'].value_counts()

In [ ]:
combined.head()

In [ ]:
combined['Purpose_Of_Loan'].value_counts()

In [ ]:
a=combined[combined['Home_Owner'].isnull()]

In [ ]:
a

In [ ]:
a.loc[a.Purpose_Of_Loan=='home_improvement']

In [ ]:
combined.loc[combined.Purpose_Of_Loan=='home_improvement','Home_Owner']='Own'

In [ ]:
combined.isnull().sum()[combined.isnull().sum()!=0]

In [ ]:
combined.loc[combined.Purpose_Of_Loan=='house','Home_Owner']='Rent'

In [ ]:
combined.isnull().sum()[combined.isnull().sum()!=0]

In [ ]:
combined.head()

In [ ]:
pd.DataFrame(combined.groupby('Length_Employed')['Annual_Income'].mean())

In [ ]:
combined.Length_Employed=combined.Length_Employed.transform(lambda x:x.fillna(x.mean()))

In [ ]:
combined.Annual_Income=combined.groupby('Length_Employed')['Annual_Income'].transform(lambda x:x.fillna(x.mean()))

In [ ]:
combined.isnull().sum()[combined.isnull().sum()!=0]

In [ ]:
combined.Home_Owner.value_counts()

In [ ]:
combined.Home_Owner=combined.Home_Owner.fillna(combined.Home_Owner.mode()[0])

In [ ]:
combined.Home_Owner.value_counts()

In [ ]:
combined.Home_Owner=combined.Home_Owner.replace(['None','Other'],['Mortgage']*2)

In [ ]:
combined.Home_Owner.value_counts()

In [ ]:
combined.info()

In [ ]:
combined.Income_Verified.value_counts()

In [ ]:
combined.Income_Verified=combined.Income_Verified.replace(['VERIFIED - income source','VERIFIED - income'],['Verified']*2)

In [ ]:
combined.Income_Verified=combined.Income_Verified.replace('not verified','Not_Verified')

In [ ]:
combined.Income_Verified.value_counts()

In [ ]:
combined.Purpose_Of_Loan.value_counts()

In [ ]:
combined.Gender.value_counts()

In [ ]:
combined.Months_Since_Deliquency.value_counts()

In [ ]:
plt.figure(figsize=(10,8))
ax=sns.heatmap(combined.corr(),annot=True,linewidths=.5,fmt='.1f')
plt.show()

In [ ]:
combined.head()

### Dropping unnecessary columns:

In [ ]:
combined.drop(['Months_Since_Deliquency','Number_Open_Accounts'],axis=1,inplace=True)

In [ ]:
combined.columns

In [ ]:
df=combined.drop('Loan_ID',axis=1)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
from sklearn.feature_extraction import FeatureHasher
fh=FeatureHasher(n_features=6,input_type='string')
hashed_feature=fh.fit_transform(df['Purpose_Of_Loan'])
hashed_feature=hashed_feature.toarray()

In [ ]:
fh=pd.DataFrame(hashed_feature)

In [ ]:
df=pd.get_dummies(df,columns=['Home_Owner','Income_Verified','Gender'],drop_first=True)

In [ ]:
dff=pd.concat([df,fh],axis=1,sort=False)
dff.shape

In [ ]:
dff.drop('Purpose_Of_Loan',axis=1,inplace=True)

In [ ]:
dff.head()

In [ ]:
dff.shape

In [ ]:
dff.head(50)

In [ ]:
dff.skew()

In [ ]:
dff.columns

### Checking distribution of numerical values:

In [ ]:
l=['Inquiries_Last_6Mo','Annual_Income']

In [ ]:
for i in l:
    sns.distplot(dff[i])
    plt.show()

### Checking outliers of numerical columns:

In [ ]:
for i in l:
    sns.boxplot(dff[i])
    plt.show()

In [ ]:
import scipy.stats as st
for i in l:
    dff[i]=list(st.boxcox(combined[i]+1)[0])
dff.skew()

In [ ]:
for i in l:
    sns.boxplot(dff[i])
    plt.show()

In [ ]:
train=dff.loc[dff['Data']=='train']

In [ ]:
train.shape

In [ ]:
test=dff.loc[dff['Data']=='test']
test.shape

In [ ]:
train=train.drop('Data',axis=1)
test=test.drop(['Data','Interest_Rate'],axis=1)

In [ ]:
train.shape,test.shape

In [ ]:
train_set.shape,test_set.shape

### Splitting train dataset into X and y:

In [ ]:
X=train.drop('Interest_Rate',axis=1)
y=train['Interest_Rate']

### Checking whether the target column is balanced or not:

In [ ]:
sns.countplot(train['Interest_Rate'])
train['Interest_Rate'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
#from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier as KNN
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from tpot import TPOTClassifier
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.3, random_state=0)

In [ ]:
pipeline_lr=Pipeline([('scalar1',StandardScaler()),
                     ('lr',LogisticRegression())])
pipeline_dt=Pipeline([('scaler2',StandardScaler()),
                     ('dt',DecisionTreeClassifier())])
pipeline_rf=Pipeline([('scalar3',StandardScaler()),
                     ('rfc',RandomForestClassifier())])
pipeline_knn=Pipeline([('scalar4',StandardScaler()),
                     ('knn',KNN())])
pipeline_xgbc=Pipeline([('scalar5',StandardScaler()),
                     ('xgboost',XGBClassifier())])
pipeline_lgbc=Pipeline([('scalar6',StandardScaler()),
                     ('lgbc',lgb.LGBMClassifier())])
pipeline_ada=Pipeline([('scalar7',StandardScaler()),
                     ('adaboost',AdaBoostClassifier())])
pipeline_sgdc=Pipeline([('scalar8',StandardScaler()),
                     ('sgradient',SGDClassifier())])
pipeline_nb=Pipeline([('scalar9',StandardScaler()),
                     ('nb',GaussianNB())])
pipeline_extratree=Pipeline([('scalar10',StandardScaler()),
                     ('extratree',ExtraTreesClassifier())])

In [ ]:
pipelines=[pipeline_lr,pipeline_dt,pipeline_rf,pipeline_knn,pipeline_xgbc,pipeline_lgbc,pipeline_ada,pipeline_sgdc,pipeline_nb,pipeline_extratree]

In [ ]:
best_accuracy=0.0
best_classifier=0
best_pipeline=""

In [ ]:
pipe_dict={0:'Logistic Regression',1:'Random Forest',2:'Decision Tree',3:'KNN',4:'XGBC',5:'LGBC',6:'ADA',7:'SGDC',8:'NB',9:'ExtraTree'}

In [ ]:
for i in pipelines:
    i.fit(X_train,y_train)
    predictions=i.predict(X_test)
    print('Classification Report : \n',(classification_report(y_test,predictions)))

In [ ]:
for i,model in enumerate(pipelines):
    print('{} Test Accuracy {}'.format(pipe_dict[i],model.score(X_test,y_test)))

In [ ]:
for i,model in enumerate(pipelines):
    if model.score(X_test,y_test)>best_accuracy:
        best_accuracy=model.score(X_test,y_test)
        best_classifier=i
        best_pipeline=model
print("Classifier with best accuracy:{}".format(pipe_dict[best_classifier]))

### Hyperparameter tuning using RandomisedSearchCV

In [ ]:
# Create a pipeline
pipe = Pipeline([("classifier", RandomForestClassifier())])

# Create dictionary with candidate learning algorithms and their hyperparameters
r_param = [ {"classifier": [RandomForestClassifier()],
             "classifier__n_estimators": [10,15,20],
             "classifier__max_depth":[15,25,30],
             "classifier__min_samples_leaf":[5,10,15,100],
             "classifier__max_leaf_nodes": [5,10]},
           
            {"classifier":[GradientBoostingClassifier()],
            "classifier__learning_rate":np.arange(0.05,0.5,0.1),
            "classifier__n_estimators":np.arange(5,10,20),
            'classifier__max_depth':np.arange(4,15),
            "classifier__min_samples_leaf":[10,15,100],
            "classifier__max_leaf_nodes": [5,10]}

      ]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
import sklearn.metrics

In [ ]:
scorer = sklearn.metrics.make_scorer(sklearn.metrics.f1_score, average = 'micro')

In [ ]:
rsearch = RandomizedSearchCV(pipe, r_param, cv=5, verbose=0,n_jobs=-1,random_state=0,scoring=scorer)

In [ ]:
rsearch.fit(X_train,y_train)

In [ ]:
print(rsearch.best_estimator_)
print("The mean accuracy of the model is through randomized search is :",rsearch.score(X_test,y_test))

In [ ]:
GBC=GradientBoostingClassifier(learning_rate=0.42000000000000004,
                                            max_depth=4, max_leaf_nodes=10,
                                            n_estimators=110)

In [ ]:
GBC_model=GBC.fit(X_train,y_train)

### Accuracy Score:

In [ ]:
accuracy_score=GBC.score(X_test,y_test)
accuracy_score

### F1 Score:

In [ ]:
pred=GBC.predict(X_test)

In [ ]:
f1_score=f1_score(pred,y_test,average='weighted')
f1_score

### Predictions of target variable of test data

In [ ]:
GBC_predictions=GBC.predict(test)
GBC_predictions.shape

In [ ]:
test.shape

In [ ]:
Submission_gbc = pd.DataFrame(GBC_predictions)

In [ ]:
Submission_gbc.to_csv('Submission_loan_gbc.csv',index=False)